Automatically Discover Missing Physics by Embedding Machine Learning into Differential Equations
================================================================================================

**Date:** [2023-07-01 Sat 11:58]



## Description



## Basic Information



## Thoughts after skimming



## Scanning Notes



### Inverse problem: data ⇒ model ?



### Load modules



In [1]:
# SciML Tools
using OrdinaryDiffEq, ModelingToolkit, DataDrivenDiffEq, SciMLSensitivity, DataDrivenSparse
using Optimization, OptimizationOptimisers, OptimizationOptimJL

# Standard Libraries
using LinearAlgebra, Statistics

# External Libraries
using ComponentArrays, Lux, Zygote, Plots, StableRNGs
using LuxCUDA                   # For GPU support

gr()                            #  gr() is a command used to select the GR backend for plotting

# Set a random seed for reproducible behaviour
rng = StableRNG(1111)

### Generate training data for predator-prey model



-   The model: $$\begin{aligned} \frac{dx}{dt} &= \alpha x - \beta x y \\ \frac{dy}{dt} &= -\delta y + \gamma x y \\ \end{aligned}$$

-   Generate data by adding some noise



In [1]:
function lotka!(du, u, p, t)
    α, β, γ, δ = p
    du[1] = α * u[1] - β * u[2] * u[1]
    du[2] = γ * u[1] * u[2] - δ * u[2]
end

# Define the experimental parameter
tspan = (0.0, 5.0)
u0 = 5.0f0 * rand(rng, 2)
p_ = [1.3, 0.9, 0.8, 1.8]       # p_ is defined here
prob = ODEProblem(lotka!, u0, tspan, p_)
solution = solve(prob, Vern7(), abstol = 1e-12, reltol = 1e-12, saveat = 0.25)

# Add noise in terms of the mean
X = Array(solution)
t = solution.t

x̄ = mean(X, dims = 2)
noise_magnitude = 5e-3
Xₙ = X .+ (noise_magnitude * x̄) .* randn(rng, eltype(X), size(X))

plot(solution, alpha = 0.75, color = :black, label = ["True Data" nothing])
scatter!(t, transpose(Xₙ), color = :red, label = ["Noisy Data" nothing])

#+attr_org: :width 1142

In [1]:
Xₙ[:,1]

2-element Vector{Float64}:
 3.1463924566781167
 1.5423300037202512

### Define UDE by combining the known terms of ODE model and Neural Network



-   Neural network part

rbf: **radial basis function** is being used as the **activation function** in a multi-layer feedforward neural network



In [1]:
rbf(x) = exp.(-(x .^ 2))

# Multilayer FeedForward
U = Lux.Chain(Lux.Dense(2, 5, rbf), Lux.Dense(5, 5, rbf), Lux.Dense(5, 5, rbf),
              Lux.Dense(5, 2))
# Get the initial parameters and state variables of the model
# device = gpu_device()
p, st = Lux.setup(rng, U)

'((layer_1 = (weight = Float32(-0.5181129 0.27094257; 0.18704869 0.76342636; … ; 0.40374994 0.013562039; -0.42771974 -0.32780463)  bias = Float32(0.0; 0.0; … ; 0.0; 0.0;;))  layer_2 = (weight = Float32(0.73459923 -0.530431 … -0.44553572 -0.19485882; -0.49543467 0.124309696 … 0.17979424 0.2070269; … ; -0.13401343 -0.114379734 … 0.75745374 -0.3036609; -0.38414848 -0.010522966 … 0.018198194 -0.29262617)  bias = Float32(0.0; 0.0; … ; 0.0; 0.0;;))  layer_3 = (weight = Float32(0.37468815 0.7608178 … 0.56425315 0.00046575852; -0.3867467 -0.39306584 … -0.643545 0.56270516; … ; -0.16022684 0.44552574 … 0.4230288 -0.16491638; -0.2221659 0.6843202 … 0.1960668 0.013009105)  bias = Float32(0.0; 0.0; … ; 0.0; 0.0;;))  layer_4 = (weight = Float32(0.48213786 0.8703153 … -0.018580398 -0.28962958; 0.14772956 -0.78825396 … 0.73837787 0.40732956)  bias = Float32(0.0; 0.0;;)))  (layer_1 = NamedTuple()  layer_2 = NamedTuple()  layer_3 = NamedTuple()  layer_4 = NamedTuple()))

-   The UDE as a dynamical system with ODE and NN parts

p\_ is defined in the experimental parameters



In [1]:
# Define the hybrid model
function ude_dynamics!(du, u, p, t, p_true)
    û = U(u, p, st)[1] # Network prediction
    du[1] = p_true[1] * u[1] + û[1]
    du[2] = -p_true[4] * u[2] + û[2]
end
# Closure with the known parameter
nn_dynamics!(du, u, p, t) = ude_dynamics!(du, u, p, t, p_)
# Define the problem
prob_nn = ODEProblem(nn_dynamics!, Xₙ[:, 1], tspan, p)

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 5.0)
u0: 2-element Vector{Float64}:
 3.1463924566781167
 1.5423300037202512

### Training loop of the UDE



-   Using *remake* as in the EnsembleProblem



In [1]:
function predict(θ, X = Xₙ[:, 1], T = t)
    _prob = remake(prob_nn, u0 = X, tspan = (T[1], T[end]), p = θ)
    Array(solve(_prob, Vern7(), saveat = T,
                abstol = 1e-6, reltol = 1e-6))
end

predict (generic function with 3 methods)

-   Loss function



In [1]:
function loss(θ)
    X̂ = predict(θ)
    mean(abs2, Xₙ .- X̂)
end

loss (generic function with 1 method)

In [1]:
Xₙ

2×21 Matrix{Float64}:
 3.14639  2.97895  2.66341  2.31933  …  3.08999  2.90096  2.56037  2.17786
 1.54233  1.81893  2.04754  2.14718     1.6282   1.90895  2.08787  2.14388

-   Track optimization by ? *callback*



In [1]:
losses = Float64[]

callback = function (p, l)
    push!(losses, l)
    if length(losses) % 50 == 0
        println("Current loss after $(length(losses)) iterations: $(losses[end])")
    end
    return false
end

#9 (generic function with 1 method)

### Optimize the weights



In [1]:
adtype = Optimization.AutoZygote()
optf = Optimization.OptimizationFunction((x, p) -> loss(x), adtype)
optprob = Optimization.OptimizationProblem(optf, ComponentVector{Float64}(p))

OptimizationProblem. In-place: true
u0: ComponentVector{Float64}(layer_1 = (weight = [-0.5181128978729248 0.2709425687789917; 0.18704868853092194 0.7634263634681702; … ; 0.403749942779541 0.013562038540840149; -0.42771974205970764 -0.3278046250343323], bias = [0.0; 0.0; … ; 0.0; 0.0;;]), layer_2 = (weight = [0.734599232673645 -0.5304309725761414 … -0.44553571939468384 -0.19485881924629211; -0.4954346716403961 0.12430969625711441 … 0.17979423701763153 0.2070268988609314; … ; -0.13401342928409576 -0.11437973380088806 … 0.7574537396430969 -0.30366089940071106; -0.3841484785079956 -0.010522966273128986 … 0.018198193982243538 -0.29262617230415344], bias = [0.0; 0.0; … ; 0.0; 0.0;;]), layer_3 = (weight = [0.37468814849853516 0.76081782579422 … 0.5642531514167786 0.0004657585232052952; -0.38674670457839966 -0.39306584000587463 … -0.6435449719429016 0.5627051591873169; … ; -0.16022683680057526 0.44552573561668396 … 0.4230287969112396 -0.16491638123989105; -0.22216589748859406 0.684320211410522

In [1]:
res1 = Optimization.solve(optprob, ADAM(), callback = callback, maxiters = 5000)
println("Training loss after $(length(losses)) iterations: $(losses[end])")

#+begin_example
Current loss after 50 iterations: 135282.17721968607
Current loss after 100 iterations: 108196.16781554442
Current loss after 150 iterations: 90728.17963490724
Current loss after 200 iterations: 77765.11037236913
Current loss after 250 iterations: 66866.41564673909
Current loss after 300 iterations: 57324.580477463234
Current loss after 350 iterations: 48902.32081932439
Current loss after 400 iterations: 41488.41486929006
Current loss after 450 iterations: 35018.874451673524
Current loss after 500 iterations: 29386.760896811047
Current loss after 550 iterations: 24505.1875746156
Current loss after 600 iterations: 20298.842729779593
Current loss after 650 iterations: 16697.555333584303
Current loss after 700 iterations: 13635.44271925403
Current loss after 750 iterations: 11050.686575435018
Current loss after 800 iterations: 8885.534145327849
Current loss after 850 iterations: 7086.37375398576
Current loss after 900 iterations: 5603.820334443037
Current loss after 950 it

-   BFGS optimization



In [1]:
optprob2 = Optimization.OptimizationProblem(optf, res1.u)
res2 = Optimization.solve(optprob2, Optim.LBFGS(), callback = callback, maxiters = 1000)
println("Final training loss after $(length(losses)) iterations: $(losses[end])")

# Rename the best candidate
p_trained = res2.u

#+begin_example
Current loss after 5050 iterations: 0.5349907614046947
Current loss after 5100 iterations: 0.5349796398261514
Current loss after 5150 iterations: 0.5348685821112326
Current loss after 5200 iterations: 0.5342704503308986
Current loss after 5250 iterations: 0.5326790144999826
Current loss after 5300 iterations: 0.528208978289582
Current loss after 5350 iterations: 0.03528077463802192
Current loss after 5400 iterations: 0.004120692192411672
Current loss after 5450 iterations: 0.0006917602309144807
Current loss after 5500 iterations: 0.00026583427446428155
Current loss after 5550 iterations: 0.0001827583508835035
Current loss after 5600 iterations: 8.578345933838757e-5
Current loss after 5650 iterations: 8.360360335123973e-5
Current loss after 5700 iterations: 7.722956976716634e-5
Current loss after 5750 iterations: 7.672005925530903e-5
Current loss after 5800 iterations: 7.521675016526402e-5
Current loss after 5850 iterations: 7.049316066349835e-5
Current loss after 5900 i

### Visualize the loss function



In [1]:
# Plot the losses
pl_losses = plot(1:5000, losses[1:5000], yaxis = :log10, xaxis = :log10,
                 xlabel = "Iterations", ylabel = "Loss", label = "ADAM", color = :blue)
plot!(5001:length(losses), losses[5001:end], yaxis = :log10, xaxis = :log10,
      xlabel = "Iterations", ylabel = "Loss", label = "BFGS", color = :red)

#+attr_org: :width 1144

### Comparison between the data and the trained network



In [1]:
## Analysis of the trained network
# Plot the data and the approximation
ts = first(solution.t):(mean(diff(solution.t)) / 2):last(solution.t)
X̂ = predict(p_trained, Xₙ[:, 1], ts)
# Trained on noisy data vs real solution
pl_trajectory = plot(ts, transpose(X̂), xlabel = "t", ylabel = "x(t), y(t)", color = :red,
                     label = ["UDE Approximation" nothing])
scatter!(solution.t, transpose(Xₙ), color = :black, label = ["Measurements" nothing])

#+attr_org: :width 1144

### The unknown term visualization



In [1]:
# Ideal unknown interactions of the predictor
Ȳ = [-p_[2] * (X̂[1, :] .* X̂[2, :])'; p_[3] * (X̂[1, :] .* X̂[2, :])']
# Neural network guess
Ŷ = U(X̂, p_trained, st)[1]

pl_reconstruction = plot(ts, transpose(Ŷ), xlabel = "t", ylabel = "U(x,y)", color = :red,
                         label = ["UDE Approximation" nothing])
plot!(ts, transpose(Ȳ), color = :black, label = ["True Interaction" nothing])

#+attr_org: :width 1144

### Putting the visualization of the training all together



In [1]:
# Plot the error
pl_reconstruction_error = plot(ts, norm.(eachcol(Ȳ - Ŷ)), yaxis = :log, xlabel = "t",
ylabel = "L2-Error", label = nothing, color = :red)
pl_missing = plot(pl_reconstruction, pl_reconstruction_error, layout = (2, 1))
pl_overall = plot(pl_trajectory, pl_missing)

#+attr_org: :width 1144

### SINDy: structural identification of dynamical systems



### Symbolic regression via sparse regression



-   Sparse symbolic regression

Sparse Symbolic Regression is a technique used in machine learning and data analysis to find simple mathematical formulas that best fit a given set of data points. It's called "sparse" because it aims to find the simplest possible formula, i.e., one with the fewest terms. "Symbolic" refers to the fact that the formula is given in terms of symbolic mathematical expressions, as opposed to, say, the weights and biases of a neural network.

-   Generate the polynomial basis



In [1]:
@variables u[1:2]
b = polynomial_basis(u, 4)
basis = Basis(b, u)

\[\begin{align}
\varphi_1 =& 1 \\
\varphi_2 =& u_1 \\
\varphi_3 =& u_1^{2} \\
\varphi_4 =& u_1^{3} \\
\varphi_5 =& u_1^{4} \\
\varphi_6 =& u_2 \\
\varphi_7 =& u_1 u_2 \\
\varphi_8 =& u_1^{2} u_2 \\
\varphi_9 =& u_1^{3} u_2 \\
\varphi_{1 0} =& u_2^{2} \\
\varphi_{1 1} =& u_2^{2} u_1 \\
\varphi_{1 2} =& u_2^{2} u_1^{2} \\
\varphi_{1 3} =& u_2^{3} \\
\varphi_{1 4} =& u_2^{3} u_1 \\
\varphi_{1 5} =& u_2^{4}
\end{align}\]

### Three core problems for sparse regressions



-   Identify the full equations from Raw data
-   Ideal situation: the missing terms are known
-   Identify the equations of the missing terms by regression on the Neural Network



### Define the three kinds of problem



In [1]:
full_problem = ContinuousDataDrivenProblem(Xₙ, t)
ideal_problem = DirectDataDrivenProblem(X̂, Ȳ)
nn_problem = DirectDataDrivenProblem(X̂, Ŷ)

Direct DataDrivenProblem{Float64} ##DDProblem#9491 in 2 dimensions and 41 samples

### Set the solver for the data driven problem



-   ADMM method and the cutoff λ



In [1]:
λ = exp10.(-3:0.01:3)
opt = ADMM(λ)

ADMM{Vector{Float64}, Float64}([0.001, 0.0010232929922807535, 0.0010471285480508996, 0.001071519305237606, 0.0010964781961431851, 0.001122018454301963, 0.0011481536214968829, 0.001174897554939529, 0.001202264434617413, 0.0012302687708123812  …  812.8305161640995, 831.7637711026708, 851.1380382023768, 870.9635899560806, 891.2509381337459, 912.0108393559096, 933.2543007969915, 954.992586021436, 977.2372209558112, 1000.0], 1.0)

-   Set options for sparse regression



In [1]:
options = DataDrivenCommonOptions(maxiters = 10_000,
                                  normalize = DataNormalization(ZScoreTransform),
                                  selector = bic, digits = 1,
                                  data_processing = DataProcessing(split = 0.9,
                                                                   batchsize = 30,
                                                                   shuffle = true,
                                                                   rng = StableRNG(1111)))

#+begin_example
DataDrivenCommonOptions{Float64, NamedTuple{(), Tuple{}}}
  maxiters: Int64 10000
  abstol: Float64 1.4901161193847656e-8
  reltol: Float64 1.4901161193847656e-8
  progress: Bool false
  verbose: Bool false
  denoise: Bool false
  normalize: DataNormalization{ZScoreTransform} DataNormalization{ZScoreTransform}()
  data_processing: DataProcessing
  roundingmode: RoundingMode{:ToZero} RoundingMode{:ToZero}()
  digits: Int64 1
  selector: bic (function of type typeof(bic))
  generate_symbolic_parameters: Bool true
  eval_expresssion: Bool false
  kwargs: NamedTuple{(), Tuple{}} NamedTuple()WARNING: both Symbolics and ModelingToolkit export "supremum"; uses of it in module DataDrivenDiffEq must be qualified
#+end_example

### Solve the full problem



In [1]:
full_res = solve(full_problem, basis, opt, options = options)
full_eqs = get_basis(full_res)
println(full_res)

"DataDrivenSolution{Float64}" with 2 equations and 6 parameters.
Returncode: Success
Residual sum of squares: 197.89962654743113
┌ Warning: Number of observations less than batch-size, decreasing the batch-size to 19
└ @ MLUtils ~/.julia/packages/MLUtils/n3C0h/src/batchview.jl:95
┌ Warning: Number of observations less than batch-size, decreasing the batch-size to 19
└ @ MLUtils ~/.julia/packages/MLUtils/n3C0h/src/batchview.jl:95

### Solve the ideal problem



In [1]:
options = DataDrivenCommonOptions(maxiters = 10_000,
                                  normalize = DataNormalization(ZScoreTransform),
                                  selector = bic, digits = 1,
                                  data_processing = DataProcessing(split = 0.9,
                                                                   batchsize = 30,
                                                                   shuffle = true,
                                                                   rng = StableRNG(1111)))

ideal_res = solve(ideal_problem, basis, opt, options = options)
ideal_eqs = get_basis(ideal_res)
println(ideal_res)

"DataDrivenSolution{Float64}" with 2 equations and 2 parameters.
Returncode: Success
Residual sum of squares: 6.130099020124461

### Solve the Neural Network Problem



In [1]:
options = DataDrivenCommonOptions(maxiters = 10_000,
                                  normalize = DataNormalization(ZScoreTransform),
                                  selector = bic, digits = 1,
                                  data_processing = DataProcessing(split = 0.9,
                                                                   batchsize = 30,
                                                                   shuffle = true,
                                                                   rng = StableRNG(1111)))

nn_res = solve(nn_problem, basis, opt, options = options)
nn_eqs = get_basis(nn_res)
println(nn_res)

"DataDrivenSolution{Float64}" with 2 equations and 5 parameters.
Returncode: Success
Residual sum of squares: 70.21200030540018

### Investigate the result of full, ideal, and neural network problem respectively



-   Full problem



In [1]:
full_eqs

\[\begin{align}
\frac{\mathrm{d}}{\mathrm{d}t} u_1 =& u_1^{2} p_1 + p_2 u_2 + u_2^{4} p_4 + u_2^{3} p_3 u_1 \\
\frac{\mathrm{d}}{\mathrm{d}t} u_2 =& u_1^{2} p_5 + p_6 u_2
\end{align}\]

In [1]:
println(full_eqs)

Model ##Basis#9508 with 2 equations
States : u[1] u[2]
Parameters : 6
Independent variable: t
Equations
Differential(t)(u[1]) = p₁*(u[1]^2) + p₂*u[2] + p₄*(u[2]^4) + p₃*(u[2]^3)*u[1]
Differential(t)(u[2]) = p₅*(u[1]^2) + p₆*u[2]

-   Ideal Problem



In [1]:
ideal_eqs

\[\begin{align}
\varphi_1 =& p_1 u_1 u_2 \\
\varphi_2 =& p_2 u_1 u_2
\end{align}\]

In [1]:
println(ideal_eqs)

Model ##Basis#9504 with 2 equations
States : u[1] u[2]
Parameters : p₁ p₂
Independent variable: t
Equations
φ₁ = p₁*u[1]*u[2]
φ₂ = p₂*u[1]*u[2]

-   Neural Network Problem



In [1]:
nn_eqs

\[\begin{align}
\varphi_1 =& p_1 + p_2 u_2 + p_3 u_1 u_2 \\
\varphi_2 =& p_4 + p_5 u_1 u_2
\end{align}\]

In [1]:
println(nn_eqs)

Model ##Basis#9512 with 2 equations
States : u[1] u[2]
Parameters : 5
Independent variable: t
Equations
φ₁ = p₁ + p₂*u[2] + p₃*u[1]*u[2]
φ₂ = p₄ + p₅*u[1]*u[2]

### Predict with NN model, since the full model failed.



-   The solution is not overlapped !



In [1]:
# Define the recovered, hybrid model
function recovered_dynamics!(du, u, p, t)
    û = nn_eqs(u, p) # Recovered equations
    du[1] = p_[1] * u[1] + û[1]
    du[2] = -p_[4] * u[2] + û[2]
end

estimation_prob = ODEProblem(recovered_dynamics!, u0, tspan, get_parameter_values(nn_eqs))
estimate = solve(estimation_prob, Tsit5(), saveat = solution.t)

# Plot
plot(solution)
plot!(estimate)

#+attr_org: :width 1144

### Fine tune the parameters by minimizing the difference between UDE and parametrized equations



-   The final parameters match the original equation



In [1]:
function parameter_loss(p)
    Y = reduce(hcat, map(Base.Fix2(nn_eqs, p), eachcol(X̂)))
    sum(abs2, Ŷ .- Y)
end

optf = Optimization.OptimizationFunction((x, p) -> parameter_loss(x), adtype)
optprob = Optimization.OptimizationProblem(optf, get_parameter_values(nn_eqs))
parameter_res = Optimization.solve(optprob, Optim.LBFGS(), maxiters = 1000)

u: 5-element Vector{Float64}:
 -0.008957235215378057
  0.00023234442439005673
 -0.8973572450573546
  0.007484513463880607
  0.7984839779240228

### Final comparison between our model and the original model



In [1]:
# Look at long term prediction
t_long = (0.0, 50.0)
estimation_prob = ODEProblem(recovered_dynamics!, u0, t_long, parameter_res)
estimate_long = solve(estimation_prob, Tsit5(), saveat = 0.1); # Using higher tolerances here results in exit of julia

In [1]:
true_prob = ODEProblem(lotka!, u0, t_long, p_)
true_solution_long = solve(true_prob, Tsit5(), saveat = estimate_long.t)
plot(estimate_long)
plot!(true_solution_long)

#+attr_org: :width 1144

### Miscellaneous post analysis



In [1]:
c1 = 3 # RGBA(174/255,192/255,201/255,1) # Maroon
c2 = :orange # RGBA(132/255,159/255,173/255,1) # Red
c3 = :blue # RGBA(255/255,90/255,0,1) # Orange
c4 = :purple # RGBA(153/255,50/255,204/255,1) # Purple

p1 = plot(t, abs.(Array(solution) .- estimate)' .+ eps(Float32),
          lw = 3, yaxis = :log, title = "Timeseries of UODE Error",
          color = [3 :orange], xlabel = "t",
          label = ["x(t)" "y(t)"],
          titlefont = "Helvetica", legendfont = "Helvetica",
          legend = :topright)

# Plot L₂
p2 = plot3d(X̂[1, :], X̂[2, :], Ŷ[2, :], lw = 3,
            title = "Neural Network Fit of U2(t)", color = c1,
            label = "Neural Network", xaxis = "x", yaxis = "y",
            titlefont = "Helvetica", legendfont = "Helvetica",
            legend = :bottomright)
plot!(X̂[1, :], X̂[2, :], Ȳ[2, :], lw = 3, label = "True Missing Term", color = c2)

p3 = scatter(solution, color = [c1 c2], label = ["x data" "y data"],
             title = "Extrapolated Fit From Short Training Data",
             titlefont = "Helvetica", legendfont = "Helvetica",
             markersize = 5)

plot!(p3, true_solution_long, color = [c1 c2], linestyle = :dot, lw = 5,
      label = ["True x(t)" "True y(t)"])
plot!(p3, estimate_long, color = [c3 c4], lw = 1,
      label = ["Estimated x(t)" "Estimated y(t)"])
plot!(p3, [2.99, 3.01], [0.0, 10.0], lw = 1, color = :black, label = nothing)
annotate!([(1.5, 13, text("Training \nData", 10, :center, :top, :black, "Helvetica"))])
l = @layout [grid(1, 2)
             grid(1, 1)]
plot(p1, p2, p3, layout = l)

#+attr_org: :width 1144